In [1]:
import nivapy3 as nivapy
import pandas as pd
from sqlalchemy import exc, text

# ICP Waters Call for Data 2022-23

## Part A: Tidy database

The aims of this notebook are to tidy existing data so that we have (i) a set of all current ICPW stations with data from 1990 to 2016, and (ii) a set of "archived" stations (from before the "TOC Trends" paper) with data up to 2012

In [2]:
eng = nivapy.da.connect()

Username:  ········
Password:  ········


Connection successful.


## 1. Tidy stations

### 1.1. Identify required changes

From the "TOC Trends" paper, we have a list of 558 ICPW stations with data from 1990 to 2016. These are the "current" ICPW stations. However, many of the station codes in the database have a `Tr18_` prefix (to distinguish them from the previous set of ICPW stations), which is no longer necessary. In addition, for the trends work we duplicated data from some of the Norwegian sites. This is no longer necessary - we should switch back to the original Norwegian stations, which are updated automatically from Aquamonitor.

The code in this section does the following:

 * Searches the database for all stations in all projects linked to ICPW. The list is saved to Excel.
 
 * The station list is reviewed manually and split into three: 
 
   * Stations that belong to ICPW in 2023 (see `all_icpw_sites_mar_2023.xlsx`). Essentially all sites linked to TOC Trends, but with the Norwegian stations switched to the originals where necessary)
   * Stations that are no longer part of the project and should therefore be "archived" to avoid confusion (worksheet `to_archive` in `icpw_stations_to_archive.xlsx`)
   * Stations with incorrect or exactly duplicated data that should be deleted (worksheet `to_delete` in `icpw_stations_to_archive.xlsx`)
   
**Note:** There is a lot of duplicated and/or mysterious data linked to old ICPW projects that I have never worked with. I am not making any changes to these, since they do not interfere with anything that I am familiar with and I assume they can be considered as "archived" without further changes.
 
 * Basic station properties for current sites are updated.
 
 * Station codes for archived sites have the suffix `_ARKIV` added.

In [3]:
# Get current stations
xl_path = r"../data/all_icpw_sites_may_2020.xlsx"
stn_df = pd.read_excel(xl_path, sheet_name="all_icpw_stns")
stn_df.head()

,station_id,station_code,station_name,latitude,longitude,altitude,continent,country,region,group
0,38115,Tr18_CA_DO1,Blue Chalk Lake,45.1999,-78.9432,344.0,North America,Canada,Ont,Trends
1,38116,Tr18_CA_DO2,Chub Lake,45.2138,-78.9836,343.0,North America,Canada,Ont,Trends
2,38117,Tr18_CA_DO3,Crosson Lake,45.0840,-79.0360,371.0,North America,Canada,Ont,Trends
3,38118,Tr18_CA_DO4,Dickie Lake,45.1510,-79.0876,379.0,North America,Canada,Ont,Trends
4,38119,Tr18_CA_DO5,Harp Lake,45.3798,-79.1335,327.0,North America,Canada,Ont,Trends


In [4]:
# Get all projects linked to ICPW
proj_df = nivapy.da.select_resa_projects(eng)
proj_df = proj_df[proj_df["project_name"].str.startswith("ICP")]

# Get all stations linked to all ICPW projects
resa_stn_df = nivapy.da.select_resa_project_stations(proj_df, eng)

# Save for checking
print(len(proj_df), "projects linked to ICPW.")
print(len(resa_stn_df), "ICPW stations found across all ICPW projects.")
resa_stn_df.to_excel("./data/all_icpw_stations_all_projects.xlsx")

598 projects in the RESA database.
71 projects linked to ICPW.
1998 ICPW stations found across all ICPW projects.


ICPW stations have been manually reviewed any any that are still used (or liable to cause confusion with those that are still used) are marked for modification in `icpw_stations_to_archive.xlsx`.

### 1.2. Delete stations

In [5]:
del_df = pd.read_excel("./data/icpw_stations_to_archive.xlsx", sheet_name="to_delete")
del_df.head()

,station_id,station_code,station_name,latitude,longitude,altitude
0,23499,CZ01_Old,"Bohemian Forest, Cerne",49.181414,13.186624,1005
1,23500,CZ02_Old,"Bohemian Forest, Certovo",49.165374,13.200272,1002
2,23501,CZ03_Old,"Bohemian Forest, Plesne",48.777810,13.867730,1086
3,23502,CZ04_Old,"Bohemian Forest, Prasilske",49.075652,13.401425,1079
4,23503,CZ05_Old,"Bohemian Forest, Laka",49.111781,13.329335,1085


In [6]:
# # Station IDs to delete
# stn_ids = del_df["station_id"].tolist()
# stn_ids = ",".join("%d" % i for i in stn_ids)

# # Get all sample IDs associated with stations
# sql = text(
#     "SELECT water_sample_id FROM resa2.water_samples "
#     "WHERE station_id IN (%s)" % stn_ids
# )
# samp_df = pd.read_sql(sql, eng)
# samp_ids = samp_df["water_sample_id"].tolist()
# samp_ids = ",".join("(1, %d)" % i for i in samp_ids)

# # Delete from sample selections
# sql = text("DELETE FROM resa2.sample_selections WHERE (1, water_sample_id) IN (%s)" % samp_ids)
# eng.execute(sql)

# # Delete from values
# sql = text("DELETE FROM resa2.water_chemistry_values2 WHERE (1, sample_id) IN (%s)" % samp_ids)
# eng.execute(sql)

# # Delete from water samples
# sql = text("DELETE FROM resa2.water_samples WHERE station_id IN (%s)" % stn_ids)
# eng.execute(sql)

# # Delete from project stations
# sql = text("DELETE FROM resa2.projects_stations WHERE station_id IN (%s)" % stn_ids)
# eng.execute(sql)

# # Delete from station par values
# sql = text("DELETE FROM resa2.stations_par_values WHERE station_id IN (%s)" % stn_ids)
# eng.execute(sql)

# # Delete from stations
# sql = text("DELETE FROM resa2.stations WHERE station_id IN (%s)" % stn_ids)
# eng.execute(sql)

### 1.3. Archive stations

In [7]:
ark_df = pd.read_excel("./data/icpw_stations_to_archive.xlsx", sheet_name="to_archive")
ark_df.head()

,station_id,station_code,station_name,latitude,longitude,altitude
0,23450,AT01,"Tirol, Schwarzsee ob Sölden",46.959667,10.941,2799.0
1,23451,BY01,Berezinsky Biosphere Reserve,54.733333,28.350,NaN
2,23452,CA01,"Ontario, Algoma Region, Batchawana",47.060000,-84.393,497.0
3,23453,CA02,"Ontario, Algoma Region, Wishart",47.041000,-84.402,388.0
4,23454,CA03,"Ontario, Algoma Region, Little Turkey",47.041000,-84.406,375.0


In [8]:
# for idx, row in ark_df.iterrows():
#     stn_id, stn_code = row["station_id"], row["station_code"]
#     stn_code += "_ARKIV"

#     # Update 'stations' table
#     sql = text(
#         "UPDATE resa2.stations "
#         "SET station_code = :stn_code "
#         "WHERE station_id = :stn_id"
#     )
#     eng.execute(sql, stn_code=stn_code, stn_id=stn_id)

### 1.4. Update current station properties

In [9]:
# Get current stations
xl_path = r"../data/all_icpw_sites_mar_2023.xlsx"
stn_df = pd.read_excel(xl_path, sheet_name="all_icpw_stns")
stn_df.head()

,station_id,station_code,nfc_code,station_name,latitude,longitude,altitude,continent,country,region,group
0,38810,AM_001,NaN,"Pambak river, 0.5 km above Khnkoyan village",40.839315,44.048911,NaN,Europe,Armenia,Armenia,Core
1,38811,AM_057,NaN,"Marmarik river, 0.5 km above Hankavan village",40.663605,44.466029,NaN,Europe,Armenia,Armenia,Core
2,38812,AM_080,NaN,"Vedi river, 0.5 km above Urtsadzor village",39.921174,44.819649,NaN,Europe,Armenia,Armenia,Core
3,38813,AM_083,NaN,"Arpa river, 0.5 km above Jermuk town",39.843045,45.686151,NaN,Europe,Armenia,Armenia,Core
4,38814,AM_089,NaN,"Meghri river, 0.5 km above Meghri town",38.915268,46.233703,NaN,Europe,Armenia,Armenia,Core


In [10]:
def upsert(stn_id, var_id, value, eng):
    """Updates a value in resa2.stations_par_values if it exists,
    otherwise inserts it as a new row.
    """
    try:
        sql = text(
            "INSERT INTO resa2.stations_par_values "
            "(station_id, var_id, value) "
            "VALUES (:stn_id, :var_id, :value)"
        )
        eng.execute(sql, stn_id=stn_id, var_id=var_id, value=value)
    except exc.IntegrityError:
        sql = text(
            "UPDATE resa2.stations_par_values "
            "SET value = :value "
            "WHERE station_id = :stn_id "
            "AND var_id = :var_id"
        )
        eng.execute(sql, stn_id=stn_id, var_id=var_id, value=value)

In [11]:
# for idx, row in stn_df.iterrows():
#     stn_id, stn_code, stn_name, lat, lon, alt, cont, country, reg, grp = row

#     # Remove 'Tr18_' prefix, if necessary
#     if stn_code.startswith("Tr18_"):
#         stn_code = stn_code[5:]

#     if pd.isna(alt):
#         alt = None

#     # Update 'stations' table
#     sql = text(
#         "UPDATE resa2.stations "
#         "SET station_code = :stn_code, "
#         "  station_name = :stn_name, "
#         "  latitude = :lat, "
#         "  longitude = :lon, "
#         "  altitude = :alt "
#         "WHERE station_id = :stn_id"
#     )
#     eng.execute(
#         sql,
#         stn_code=stn_code,
#         stn_name=stn_name,
#         lat=lat,
#         lon=lon,
#         alt=alt,
#         stn_id=stn_id,
#     )

#     # Update 'stations_par_values'
#     upsert(stn_id, 322, cont, eng)
#     upsert(stn_id, 261, country, eng)
#     upsert(stn_id, 254, reg, eng)
#     upsert(stn_id, 357, grp, eng)

### 1.5. Update projects

Create a project with all the current stations and delete the old project named `ICPW_All_Stations_2020`.

In [12]:
# # Delete project 'ICPW_All_Stations_2020'
# sql = text("DELETE FROM resa2.projects_stations WHERE project_id = 4510")
# eng.execute(sql)

# sql = text("DELETE FROM resa2.projects WHERE project_id = 4510")
# eng.execute(sql)

In [13]:
# Created new project with ID 4617
proj_stn_df = stn_df[["station_id"]].copy()
proj_stn_df["project_id"] = 4617
proj_stn_df.head()

,station_id,project_id
0,38810,4617
1,38811,4617
2,38812,4617
3,38813,4617
4,38814,4617


In [14]:
# proj_stn_df.to_sql(
#     name="projects_stations",
#     schema="resa2",
#     con=eng,
#     if_exists="append",
#     index=False,
# )